In [1]:
# provide a filename for the input text:
filename = 'KR6a0001.txt'

# provide a filename for the dictionary (code assumes one chinese term per line)
vocabulary_file = 'dictionary_terms.txt'

# these were further down below, but I moved them up since that's basically the main this file does

In [2]:
import re,itertools,time
# import stanza # if you want to use the stanza tokenizer
#note there are other lines below that you have to uncomment to use as well

In [3]:
# this begins by defining various functions to break up a longer text into paragraphs, sentences, and phrases
# split a text into paragraphs based on double line breaks
def graph_tokenizer(text):
    return text.split("\n\n")

In [4]:
# break into sentences 
def sent_tokenizer(text):
    return re.split(r'[。？！\．]', text)

In [5]:
# break into phrases (there is some duplication)
def phrase_tokenizer(text):
    return re.split(r'[。？！；：；、，「」『』《》 *  ()○]', text)

In [6]:
# flatten a two-layer list into a single layer list
# i'm not sure what this does
def flatten(input_list):
    new_list = list(itertools.chain(*input_list))
    return [l for l in new_list if l != ""]

In [7]:
# remove unnecessary content. This is based on texts found in the kanseki 
# repository
# not sure this wil be necessary for me
def clean_up(text):
    # remove structural markup
    text = re.sub(r'<.+?>', '', text)
    # remove commented lines
    text = re.sub(r'#.+', '', text)
    # remove other characters
    rem = ["\u3000", '\n', '¶']
    for r in rem:
        text = text.replace(r, '')
    return text

In [8]:
# instantiate tokenzier if using stanza. if you prefer you can just assume 
# characters not in your dictionary are single words, as I have done here
# nlp = stanza.Pipeline('zh-hant', processors='tokenize', use_gpu=True)

In [10]:
# open your dictionary (in my case this is just a plain text)
with open(vocabulary_file, 'r', encoding='utf8') as rf:
    vocab = list(set(rf.read().split("\n")))

#print(vocab) #for testing

['多陀', '愛身天', '三根', '七百結集', '三衰', '芻摩', '富', '六根功德', '善歲部', '不空', '八萬四千', '木槵子', '親眷', '奴隸', '沒劫', '留', '修摩那', '一色', '五惡', '栗底尼迦耶', '應化利生', '五法', '漏盡明', '甘露咒', '大乘經', '華藏', '唯識', '鄔波塞', '般羅颯迷', '心佛與衆生是三無差別', '倶生', '迦葉遺', '頡', '騎驢覓驢', '雜修', '嗣', '觀法', '夫婦', '齋鼓', '歌詠', '四仙', '浄', '那術', '法顯', '龍女', '師孫', '惟一眞實', '母親', '惜', '寶積', '垢汗', '施羅', '兜率', '圓覺經', '慧燈王', '千眼千臂觀世音', '不退菩薩', '究牟陀', '綺', '疑結', '乞灑', '二現', '四大弟子', '優陀伽', '正覺', '去來實有宗', '七加行', '難思', '上首', '勿伽', '大聖曼殊室利', '二十五才便', '定善', '語', '六蔽', '劫比羅伐窣都', '流來生死', '識無處邊', '解境十佛', '鳳', '五不正食', '摩訶曼殊沙華', '世無障礙智戒', '梵德', '下', '者', '斡', '提羅', '性念處', '阿耶羅', '文殊院', '舍利沙婆', '空諦', '三昧耶', '擇法眼', '摩咥哩迦', '阿闍黎', '德甁', '沙羅', '薩婆多論', '阿濕婆', '薜茘', '文眞鄰陀', '上品', '變化人', '空海', '理性', '曼供', '夏竟', '出隊', '教內', '槃頭', '散業念佛', '百界', '善現天', '兩垢', '入骨', '太麤生', '釋女', '迦攝波', '禪法', '三味', '菩提索多', '阿羅訶', '化法教', '能信', '羯羅尾羅', '波抳', '轉識', '沙門', '佛菩薩化生', '大明王', '摩虎羅', '隱形', '卑慢', '二定', '五乘居天', '明智', '邪山', '瞎', '屈陀迦阿含', '五音', '題', '四句執', '彌勤', '等覺', '理卽', '剃髮', '繫著', '堅那

In [19]:
# sort the vocabulary by length, to prioritize long terms
vocab = sorted(vocab, key=len, reverse=True)
# filter out any blank entries
vocab = [v for v in vocab if v != ""] #this basically removes any blank entries i.e. "", but idk why this would be a problem; i guess it's just a data cleaning thing
#print(len(vocab))
#print(vocab)


In [35]:
# compile the regular expression to match any given vocabulary word, capturing 
# the result

#my_regex = re.compile("(" + ("|").join(vocab) + ")")　 #()" + ("|").join(vocab) + ") #idk why this wasn't working
my_regex = re.compile("(" + ("|").join(vocab) + ")") # but this does work?? why? 

#I'm not too sure what is going on here, looksl ike maybe all the vocab words are being re-compiled into another list of regexes

# print(my_regex) #for testing

re.compile('(手千眼觀世音菩薩大廣大圓滿無礙大悲心陀羅尼經|金剛頂瑜伽千手千眼觀自在菩薩修行儀軌經|千手千眼廣大圓滿無礙大悲心陀羅尼經|啞曷囉啞曷囉馬麻藹由而傘塔囉尼|千手千眼觀世音菩藤姥陀羅尼身經|一切如來諸法本性淸淨蓮華三昧|比丘尼僧祇律波羅提木叉戒經|一切如來眼色如明照三摩地|一切女人皆是衆惡之所住處|千手千眼大慈大悲觀音菩薩|佛說傘蓋佛母總持陀羅尼經|父母所生之肉身卽爲佛身也|漸漸修習而次第開顯佛德)


In [37]:
# start a timer (not necessary, just for curiosity's sake)
st=time.time()

print(st) #also not necessary, just for curiosity's sake

1589826804.32973


In [ ]:
# open the file to tokenize
with open(filename) as rf:
    text = rf.read()

# Here I reduce the document into individual paragraphs. this is not necessary
# stricktly speaking, but helps when working with long texts
paragraphs = graph_tokenizer(text)

# Clean the paragraphs and then find the dictionary words, marking up with <w>
# tags
paragraphs = [my_regex.sub(r'<w>\1</w>', clean_up(p)) for p in paragraphs] # the clean_up removes stuff from the kanseki database format, so it's not strictly necessary

# Break into sentences. Another step that is not super necessary, but results
# in a nice output (sentences divided into words)
sentences = [sent_tokenizer(p) for p in paragraphs]# not sure why this wasnt before he did the regex... but ok why not

# flatten the list from a list of paragraphs, each of which is a list of sentences
# to just a list of sentences
sentences = flatten(sentences) 

# remove any sentence that has no words in it
sentences = [s for s in sentences if s != ""]

# create an empty list to contianed the parsed output
parsed = []

# iterate through each sentence and parse it as necessary
for sentence in sentences:

    # keep track of words in the current sentence
    temp_parse = []

    # futher reduce the text into phrases, as words will not cross phrase boundaries
    for phrase in phrase_tokenizer(sentence):

        # keep track of the words in the current phrase
        temp_words = []
        
        # find the first marked up word
        word_start = phrase.find("<w>")

        # as long as there are still marked up words, keep searching
        while word_start != -1:
            # if an identified word is at the beginning of the paragraph then append it to
            # the parsed list
            if word_start == 0:
                word_end = phrase.find("</w>")
                temp_words.append(phrase[word_start+3:word_end])
                # move the text forward past the marked-up word
                phrase =phrase[word_end+4:]
            else:
                # specify the chunk up to the next identified word
                chunk = phrase[:word_start]
                # if the chunk is just one character, then append it to the list
                if len(chunk) == 1:
                    temp_words.append(chunk)
                # otherwise, use the alternative parser method and EXTEND the list 
                # assuming the alternative parser returns a list of words
                else:
                    temp_words.extend(list(chunk))
                    # alternatively you could switch this for:
                    #temp_words.extend([w.text for w in nlp(chunk).sentences[0].words])
                    
                # move the text up to the next identified word
                phrase = phrase[word_start:]
            # search for the next identified word. Once there are no more words,
            # you will exit out of the while loop.
            word_start = phrase.find("<w>")

        # there is a possiblity that there are no more marked up words, but still 
        # untokenized text!

        # check if there is remaining text
        remaining = len(phrase)

        # if there is remaining text, then parse it
        if remaining > 0:
            # if it is just one character save it
            if remaining == 1:
                temp_words.append(phrase)
            # otherwise, use a parser to extend the list
            else:
                temp_words.extend(list(phrase))
                # alternatively you could switch this for:
                #temp_words.extend([w.text for w in nlp(phrase).sentences[0].words])
        temp_parse.extend(temp_words)
    
    # as long as there are words in the temp_parse list, then append it to the
    # parsed list
    if len(temp_parse) > 0:
        parsed.append(temp_parse)    

# print the first ten sentences
print(parsed[:10])

# finish the timer:
et = time.time()
print(f"This operation took {et-st:.2f} seconds")